In [1]:
import tensorflow as tf
import keras
import os
import shutil
import cv2
import numpy as np

import glob as glob
import matplotlib.pyplot as plt
from keras.models import load_model
import segmentation_models_3D as sm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

focal_loss = sm.losses.CategoricalFocalLoss()


Segmentation Models: using `tf.keras` framework.


# Evaluating on Test Data

In [48]:
test_dir = "Evaluation/Test"
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    batch_size=16,
    image_size=(256, 256),
    seed=123,
    shuffle=False,
    interpolation='area'
)

normalization_layer = tf.keras.layers.Rescaling(1./255)
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


Found 394 files belonging to 4 classes.


In [4]:
vgg19 = load_model("model_checkpoints/vgg19_cc_loss.hdf5")
densenet121 = load_model("model_checkpoints/densenet121_focal_loss.hdf5", custom_objects={'focal_loss': focal_loss})
mobilenetv2 = load_model("model_checkpoints/mobilenetv2_focal_loss.hdf5",  custom_objects={'focal_loss': focal_loss})



In [7]:
predictions = vgg19.predict(test_ds)


25/25 [==============================] - 25s 512ms/step


In [9]:
predictions[0]

array([8.7144428e-05, 3.3264086e-04, 5.8504579e-06, 9.9957436e-01],
      dtype=float32)

# Create an Ensemble model

In [11]:
models = [vgg19, densenet121, mobilenetv2]


In [ ]:
y_test =  np.array([])
for x, y in test_ds:
  print(y.shape)
  y_test = np.concatenate([y_test, y.numpy()])

In [51]:
y_test.shape
y_test[0]

0.0

In [85]:
preds = [model.predict(test_ds) for model in models]
preds = np.array(preds)
weights = [0.4, 0.3, 0.3]


25/25 [==============================] - 1s 40ms/step


In [58]:
preds = preds.argmax(axis=-1)

In [65]:
preds.shape

(3, 394, 4)

In [20]:
pip install scikit-learn

     ---------------------------------------- 9.2/9.2 MB 3.6 MB/s eta 0:00:00
     -------------------------------------- 302.2/302.2 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Weighted Ensemble approach

In [86]:
from sklearn.metrics import accuracy_score

weighted_preds = np.tensordot(preds, weights, axes=((0),(0)))
weighted_ensemble_prediction = np.argmax(weighted_preds, axis=1)

weighted_accuracy = accuracy_score(y_test, weighted_ensemble_prediction)

In [87]:
print(weighted_accuracy)

0.9873096446700508


In [75]:
weighted_preds.shape

(394, 4)

In [28]:
preds

0.27411167512690354


In [78]:
prediction1 = vgg19.predict(test_ds)
prediction1 = prediction1.argmax(axis=-1)

prediction2 = densenet121.predict(test_ds)
prediction2 = prediction2.argmax(axis=-1)

prediction3 = mobilenetv2.predict(test_ds)
prediction3 = prediction3.argmax(axis=-1)

25/25 [==============================] - 1s 40ms/step


In [53]:
prediction1.shape

(394,)

In [82]:
accuracy1 = accuracy_score(y_test, prediction1)
accuracy2 = accuracy_score(y_test, prediction2)
accuracy3 = accuracy_score(y_test, prediction3)

In [83]:
print(accuracy1)
print(accuracy2)
print(accuracy3)

0.9847715736040609
0.9873096446700508
0.9746192893401016


# Weighted Ensemble using Grid Search

In [ ]:
import pandas as pd
df = pd.DataFrame([])
max_acc = []
for w1 in range(0, 5):
    for w2 in range(0,5):
        for w3 in range(0,5):
            wts = [w1/10.,w2/10.,w3/10.]
            wted_preds1 = np.tensordot(preds, wts, axes=((0),(0)))
            wted_ensemble_pred = np.argmax(wted_preds1, axis=1)
            weighted_accuracy = accuracy_score(y_test, wted_ensemble_pred)
            df2 = pd.DataFrame({'wts1':wts[0], 'wts2':wts[1], 'wts3':wts[2], 'acc':weighted_accuracy})
            df.append(df2)
            max_acc.append(weighted_accuracy)



In [114]:
max_acc= np.array(max_acc)
max_acc.shape
prediction = np.argmax(max_acc)
print(max_acc.shape)

(125,)


In [115]:
print(max_acc[prediction])

# Test accuracy improved by 0.2% from the best model

0.9898477157360406
